# Python sqlite3 Tutorials

In [1]:
##%reload_ext load_style
##%load_style ../custom/talk.css

## Check environment

In [2]:
import sqlite3
sqlite3.version

'2.6.0'

In [3]:
sqlite3.sqlite_version

'3.13.0'

### Example code using sqlite3 

In [4]:
import sqlite3 as lite
import sys

con = None
db_filepath = 'test.db'

try:
    con = lite.connect(db_filepath)
    cur = con.cursor()    
    cur.execute('SELECT SQLITE_VERSION()')
    
    data = cur.fetchone()
    print("SQLite version: {}".format(data)) 
    
except lite.Error as e:
    print("Error {}:".format(e.args[0]))
    sys.exit(1)
    
finally:
    if con:
        con.close()

SQLite version: ('3.13.0',)


### Create test.db 
- **test.db** in subdirectory 'src\db'

In [5]:
import sqlite3 as lite
import sys
con = lite.connect('data//test.db')

with con:
    cur = con.cursor()    
    cur.execute("CREATE TABLE Cars(Id INT, Name TEXT, Price INT)")
    cur.execute("INSERT INTO Cars VALUES(1,'Audi',52642)")
    cur.execute("INSERT INTO Cars VALUES(2,'Mercedes',57127)")
    cur.execute("INSERT INTO Cars VALUES(3,'Skoda',9000)")
    cur.execute("INSERT INTO Cars VALUES(4,'Volvo',29000)")
    cur.execute("INSERT INTO Cars VALUES(5,'Bentley',350000)")
    cur.execute("INSERT INTO Cars VALUES(6,'Citroen',21000)")
    cur.execute("INSERT INTO Cars VALUES(7,'Hummer',41400)")
    cur.execute("INSERT INTO Cars VALUES(8,'Volkswagen',21600)")

In [6]:
con.commit()
con.close()

## List files in a given directory

In [7]:
from os import listdir
from os.path import isfile, join

def listFiles(path):
    onlyfiles = []

    for f in listdir(dbpath):
        if isfile(f):
            onlyfiles.append(f)

    # OR just a single statement:
    #onlyfiles = [f for f in listdir(dbpath) if isfile(join(dbpath, f))]

    print(onlyfiles)

In [8]:
dbpath = 'data'
listFiles( dbpath )

['test.db']


In [9]:
!ls ./data -lR

./data:
total 8
-rw-r--r-- 1 ubuntu ubuntu 8192 Nov 12 16:59 test.db


In [10]:
# !dir src\db\*.db /B/A     &REM Windows CLI command

## Retrieving data

In [11]:
# db_filepath = 'src\\db\\test.db'      # for Windows
db_filepath = 'data/test.db'

In [12]:
import sqlite3 as lite
import sys

con = lite.connect( db_filepath )
with con:    
    cur = con.cursor()    
    cur.execute("SELECT * FROM Cars")

    rows = cur.fetchall()  # a Tuple of Tuples as the execution of SQL
    for row in rows:       # row is a Tuple
        print(row)
con.close()

(1, 'Audi', 52642)
(2, 'Mercedes', 57127)
(3, 'Skoda', 9000)
(4, 'Volvo', 29000)
(5, 'Bentley', 350000)
(6, 'Citroen', 21000)
(7, 'Hummer', 41400)
(8, 'Volkswagen', 21600)


## The dictionary cursor

The default cursor returns the data in **a tuple of tuples**. When we use a dictionary cursor, the data is sent in the form of Python dictionaries. This way we can refer to the data by their column names.

In [13]:
import sqlite3 as lite

con = lite.connect( db_filepath )    
with con:
    con.row_factory = lite.Row
    
    cur = con.cursor() 
    cur.execute("SELECT * FROM Cars")

    rows = cur.fetchall()
    for row in rows:
        print( "{:3} {:20s} {:>10,.2f}".format(
                row["Id"], row["Name"], row["Price"]))
        
con.close()

  1 Audi                  52,642.00
  2 Mercedes              57,127.00
  3 Skoda                  9,000.00
  4 Volvo                 29,000.00
  5 Bentley              350,000.00
  6 Citroen               21,000.00
  7 Hummer                41,400.00
  8 Volkswagen            21,600.00


## Parameterized queries

When we use **parameterized queries**, we use *placeholders* instead of directly writing the values into the statements.
- Parameterized queries increase security and performance.
- The Python sqlite3 module supports two types of placeholders: **question marks** and **named placeholders**.

In [14]:
import sqlite3 as lite
import sys

uId = 1
uPrice = 62300 
con = lite.connect(db_filepath)
with con:
    cur = con.cursor()    

    cur.execute("UPDATE Cars SET Price=? WHERE Id=?", (uPrice, uId))        
    con.commit()
    
    print("Number of rows updated: {}".format(cur.rowcount))

con.commit()
con.close()

Number of rows updated: 1


We update a price of one car. In this code example, we use the **question mark** placeholders.

```sql
cur.execute("UPDATE Cars SET Price=? WHERE Id=?", (uPrice, uId)) 
```

The question marks `?` are placeholders for values. The values are added to the placeholders. 

The `rowcount` property returns the number of updated rows. In our case one row was updated.

## Getting column names

In [15]:
import sqlite3 as lite
con = lite.connect( db_filepath )
cursor = con.execute('SELECT * FROM Cars')

names = list(map(lambda x: x[0], cursor.description))
con.close()

names

['Id', 'Name', 'Price']

## Def listDbSchema

In [16]:
import sqlite3 as lite 

def listDbSchema(db_filepath):
    """ Print the schemas of Tables in a database with athe given file path """
    con = lite.connect(db_filepath)
    cur = con.cursor()
    
    sql_lsTables = "SELECT name FROM sqlite_master WHERE type='table';"
    cur.execute(sql_lsTables)
    table_names = cur.fetchall()
    
    for name in table_names:
        s = "PRAGMA table_info({:s})".format(name[0])
        cur.execute(s)
        tableinfo = cur.fetchall()
        print('{} Table: {:^15} {}'.format('-'*15, name[0], '-'*15))
        for row in tableinfo:
            print("{!s:3} {!s:16}{!s:12}{!s:>4} {!s:10} {!s:>4}".format(*row))
            
    con.close()

In [17]:
# example using listDbSchema()

listDbSchema( db_filepath )

--------------- Table:      Cars       ---------------
0   Id              INT            0 None          0
1   Name            TEXT           0 None          0
2   Price           INT            0 None          0


## SQLite - Useful Functions

> SQLite documentation : 
>  - [SQLite Functions](https://www.sqlite.org/c3ref/funclist.html)
>  - [Core Functions](https://www.sqlite.org/lang_corefunc.html)

In [18]:
import sqlite3 as lite
import sys

con = lite.connect(db_filepath)
with con:    
    cur = con.cursor()
    sql = "SELECT max(price) FROM Cars"
    cur.execute(sql)
    price_max = cur.fetchall()

    sql = "SELECT min(price) FROM Cars"
    cur.execute(sql)
    price_min = cur.fetchall()
    
    sql = "SELECT avg(price) FROM Cars"
    cur.execute(sql)
    price_avg = cur.fetchall()

    sql = "SELECT count(price) FROM Cars"
    cur.execute(sql)
    price_cnt = cur.fetchall()
    
    print("Car Prices:-----------")
    print( "Min  : {:>10,.2f} ".format(price_min[0][0]))
    print( "Max  : {:>10,.2f} ".format(price_max[0][0]))
    print( "Avg  : {:>10,.2f} ".format(price_avg[0][0]))
    print('-'*22)
    print( "Count: {:>10} ".format(price_cnt[0][0]))

con.close()

Car Prices:-----------
Min  :   9,000.00 
Max  : 350,000.00 
Avg  :  73,928.38 
----------------------
Count:          8 


## Def listDbFiles

In [19]:
import sqlite3 as lite

def listDbFiles(db_filepath):
    con = lite.connect(db_filepath)
    cur = con.cursor()
    cur.execute("PRAGMA database_list")
    rows = cur.fetchall()
    con.close()

    for row in rows:
        print("{!s:4} {!s:16} {}".format(*row))

In [20]:
listDbFiles( db_filepath )

0    main             /home/ubuntu/workspace/notebooks/data/test.db


## Def listMaster

In [21]:
import sqlite3 as lite

def listMaster(db_filepath):
    con = lite.connect(db_filepath)
    cur = con.cursor()
    cur.execute("PRAGMA table_info('sqlite_master')")
    rows = cur.fetchall()
    con.close()

    for row in rows:
        print("{:4} {:16} {:4}".format(*row))

In [22]:
listMaster( db_filepath )

   0 type             text
   1 name             text
   2 tbl_name         text
   3 rootpage         integer
   4 sql              text


## Def listForeignKeys()

In [23]:
import sqlite3 as lite

def listForeignKeys(db_filepath, table_name):
    con = lite.connect(db_filepath)
    cur = con.cursor()
    
    s= "PRAGMA foreign_key_list({})".format(table_name)
    cur.execute( s )
    rows = cur.fetchall()
    con.close()

    pHead_str = "Database: {}, Table {}: {}"
    pDetail_str = "{!s:4}{!s:4} {!s:15} {!s:15} {!s:15} {!s:10} {!s:10} {!s:10}"
    print(pHead_str.format(db_filepath, table_name, '-'*42))
    if (len(rows) > 0):
        for row in rows:
            print(pDetail_str.format(*row))
    else:
        print("--- no foreign key ---")

In [24]:
listForeignKeys(db_filepath, 'Prices')

Database: data/test.db, Table Prices: ------------------------------------------
--- no foreign key ---


## Def listTables()

In [25]:
import sqlite3 as lite

def listTables(db_filename):
    sql = "SELECT name, sql FROM sqlite_master WHERE type='table' ORDER BY name;"

    con = lite.connect(db_filename)
    cur = con.cursor()
    cur.execute(sql)
    rows = cur.fetchall()
    con.close()

    for row in rows:
        print("{:4} {:16}".format(*row))

In [26]:
listTables( db_filepath )

Cars CREATE TABLE Cars(Id INT, Name TEXT, Price INT)


## Execute sql from a file

In [27]:
from sqlite3 import OperationalError

def executeSQLScript(connection, filename):
    # Open and read the file as a single buffer
    fd = open(filename, 'r')
    sqlFile = fd.read()
    fd.close()

    c = connection.cursor()
    c.execute("PRAGMA foreign_keys = ON")

    # all SQL commands (split on ';')
    sqlCommands = sqlFile.split(';')

    # Execute every command from the input file
    for command in sqlCommands:
        # This will skip and report errors
        # For example, if the tables do not yet exist, this will skip over
        # the DROP TABLE commands
        try:
            c.execute(command)
        except OperationalError as msg:
            print("Command skipped: {!s}".format(msg))
    con.commit()

## Create example2_products.db

In [29]:
import sqlite3 as lite
import os

example2_db_filename = "src/db/example2_products.db"
con = lite.connect(example2_db_filename) 

sql_filepath = 'src/create_table_example2.sql'
executeSQLScript(con, sql_filepath)

con.close()

In [30]:
listFiles( dbpath )

['test.db']


In [31]:
# Count the rows in Tables Company, Product 
import sqlite3 as lite

con = lite.connect( example2_db_filename )
cur = con.cursor()
with con:    
    sql = "SELECT count(name) FROM Company"
    cur.execute(sql)
    company_cnt = cur.fetchall()
    sql = "SELECT count(name) FROM Product"
    cur.execute(sql)
    product_cnt = cur.fetchall()
    
    print("Table Company :-----------")
    print("         Count: {:>10} ".format(company_cnt[0][0]))

    print("Table Product :-----------")
    print("         Count: {:>10} ".format(product_cnt[0][0]))
    
con.close()

Table Company :-----------
         Count:          4 
Table Product :-----------
         Count:          5 


In [32]:
con = lite.connect( example2_db_filename )
print(con)

# To get the file path of a database
cur = con.cursor()
cur.execute("PRAGMA database_list")
rows = cur.fetchall()

for row in rows:
    print(row[0], row[1], row[2])

0 main /home/ubuntu/workspace/notebooks/src/db/example2_products.db


*The third parameter* (`row[2]`) is the file name of the database.

In [33]:
con.close()

In [34]:
listTables(example2_db_filename)

Company CREATE TABLE Company
  (name varchar(20) primary key,
   country varchar(20),
   no_employees int,
   for_profit char(1)
   )
Product CREATE TABLE Product
  (name varchar(20) primary key,
   price float,
   category varchar(20),
   manufacturer varchar(20), 
   FOREIGN KEY(manufacturer) REFERENCES Company(name) ON UPDATE CASCADE
   )


In [35]:
example2_db_filename = "example2_products.db"

In [36]:
listDbSchema(example2_db_filename)

In [37]:
listForeignKeys(example2_db_filename, 'Product')

Database: example2_products.db, Table Product: ------------------------------------------
--- no foreign key ---


## Retrieving data

In [38]:
example2_db_filename = "src/db/example2_products.db"

In [39]:
import sqlite3 as lite

sql = """SELECT name, price, manufacturer 
FROM Product
WHERE category = 'gadget'"""

con = lite.connect(example2_db_filename)
cur = con.cursor()    
cur.execute(sql)

rows = cur.fetchall()
if len(rows) == 0:
    print("[{}] No {} records.".format(example2_db_filename, 'Product'))
else:
    for row in rows:
        print(row)
        
con.close()

('Gizmo', 19.99, 'GizmoWorks')
('PowerGizmo', 29.99, 'Hitachi')
('SuperGizmo', 49.99, 'Charity')


## Table Views

```sql
CREATE VIEW ... 
```

In [40]:
import sqlite3 as lite

con = lite.connect(db_filepath)    
with con:
    sql = """CREATE VIEW view_cars AS 
    SELECT Id, Name FROM Cars"""
    
    cur = con.cursor() 
    cur.execute(sql)
        
con.close()

In [41]:
import sqlite3 as lite

def listViews(db_filename):
    sql = "SELECT name, sql FROM sqlite_master WHERE type='view' ORDER BY name;"

    con = lite.connect(db_filename)
    cur = con.cursor()
    cur.execute(sql)
    rows = cur.fetchall()
    con.close()

    for row in rows:
        print("{:4} {:16}".format(*row))

In [42]:
listViews(db_filepath)

view_cars CREATE VIEW view_cars AS 
    SELECT Id, Name FROM Cars


In [43]:
import sqlite3 as lite

con = lite.connect(example2_db_filename)    
with con:
    sql = """CREATE VIEW view_product_gadget AS 
    SELECT name, price, manufacturer FROM Product WHERE category = 'gadget'"""
    
    cur = con.cursor() 
    cur.execute(sql)
        
con.close()

In [44]:
listViews(example2_db_filename)

view_product_gadget CREATE VIEW view_product_gadget AS 
    SELECT name, price, manufacturer FROM Product WHERE category = 'gadget'


## Saving data schema to CSV using Pandas

In [45]:
import pandas as pd
con = sqlite3.connect('src/db/test.db')

table_name = "Cars"
sql = "SELECT * from %s" % (table_name)
table = pd.read_sql_query(sql, con)
print(table)
table.to_csv(table_name + '.csv', index_label='index')

con.close()

   Id        Name   Price
0   1        Audi   62300
1   2    Mercedes   57127
2   3       Skoda    9000
3   4       Volvo   29000
4   5     Bentley  350000
5   6     Citroen   21000
6   7      Hummer   41400
7   8  Volkswagen   21600


In [46]:
import sqlite3
import pandas as pd

def listTablesViewsToCsv(db_filepath):
    con = sqlite3.connect(db_filepath)
    cursor = con.cursor()
    
    sql = "SELECT name FROM sqlite_master WHERE type='table' or type='view';"
    cursor.execute(sql)
    tables = cursor.fetchall()
    
    for table_name in tables:
        table_name = table_name[0]
        table = pd.read_sql_query("SELECT * from %s" % table_name, con)
        table.to_csv(table_name + '.csv', index_label='index')

In [47]:
listTablesViewsToCsv('src//db//test.db')

# Cache.db

In [48]:
con = lite.connect('cache.db', timeout=10)
con.close()

###  end of IPynb